# 欢迎来到香山 Tutorial - GEM5 部分

## 1. 编译运行 GEM5

In [ ]:
# 由于服务器资源有限，我们为大家已经编译好了 gem5 和 DRAMSim3，建议大家跳过这一部分。

pushd ../../ && source env.sh && popd

check_env() {
    if [[ -z "${gem5_home}" ]]; then
        echo "Error: No gem5_home found" 1>&2
        return 1
    fi
}

prepare_gem5() {
    pushd $gem5_home && \
    cd ext/dramsim3 && \
    (stat DRAMsim3 || git clone https://github.com/umd-memsys/DRAMSim3.git DRAMsim3) && \
    cd DRAMsim3 && mkdir -p build && cd build && cmake .. && make -j `nproc` && \
    popd
}

build_gem5() {
    pushd $gem5_home && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    popd
}

build_nemu_diff() {
    # Used for difftest and GCPT restorer
    pushd $gem5_home && \
    wget https://github.com/OpenXiangShan/GEM5/releases/download/2024-10-16/riscv64-nemu-interpreter-c1469286ca32-so && \
    popd
}

prepare_data_proc() {
    # Validated commit for tutorial: 4000c092b8bde21fd4aa493f9907fa100dbcb3fc
    (stat gem5_data_proc || git clone https://github.com/shinezyy/gem5_data_proc.git) && \
    pushd gem5_data_proc && \
    git reset --hard 4000c092b8bde21fd4aa493f9907fa100dbcb3fc && \
    pip3 install -r requirements.txt && \
    popd
}

# check_env && prepare_gem5 && build_gem5 && build_nemu_diff && prepare_data_proc

# 由于服务器资源有限，我们为大家已经编译好了 gem5 和 DRAMSim3，建议大家跳过这一部分。如果你在自己的机器上执行，请去除上一行注释

## 2. 运行 CoreMark

In [ ]:
%%bash
#!/usr/bin/env bash

pushd ../../ && source env.sh && popd

pushd $gem5_home && \
export LD_LIBRARY_PATH=$gem5_home/ext/dramsim3/DRAMsim3:$LD_LIBRARY_PATH && \
export GCBV_REF_SO=$gem5_home/riscv64-nemu-interpreter-c1469286ca32-so && \
mkdir -p util/xs_scripts/coremark && \
cd util/xs_scripts/coremark && \
$gem5_home/build/RISCV/gem5.opt $gem5_home/configs/example/xiangshan.py \
--raw-cpt --generic-rv-cpt=$NOOP_HOME/ready-to-run/coremark-2-iteration.bin && \
popd

## 3. 查看 TopDown 计数器

In [5]:
%%bash
#!/usr/bin/env bash

pushd ../../ >/dev/null && source env.sh >/dev/null && popd >/dev/null

pushd gem5_data_proc && \
python3 batch.py \
  -s $gem5_home/util/xs_scripts/coremark \
  -t --topdown-raw && \
popd

~/tutorial-2025/tutorial/p7-xs-gem5/gem5_data_proc ~/tutorial-2025/tutorial/p7-xs-gem5
workload: m5out, point: 0, segments: 1
[('m5out_0', '/home/cyy/tutorial-2025/gem5/util/xs_scripts/coremark/m5out/stats.txt')]
Process finished job: m5out_0
/home/cyy/tutorial-2025/gem5/util/xs_scripts/coremark/m5out/stats.txt
{'m5out_0': {'OtherStall': 0, 'CommitSquash': 143586, 'ResumeUnblock': 0, 'OtherMemStall': 516, 'Atomic': 0, 'MemCommitRateLimit': 0, 'MemNotReady': 3494111, 'MemSquashed': 0, 'StoreMemBound': 0, 'StoreL3Bound': 0, 'StoreL2Bound': 0, 'StoreL1Bound': 20811, 'LoadMemBound': 6, 'LoadL3Bound': 0, 'LoadL2Bound': 0, 'LoadL1Bound': 190084, 'InstNotReady': 2740, 'SerializeStall': 39191, 'InstSquashed': 41700, 'InstMisPred': 0, 'FetchBufferInvalid': 0, 'SquashStall': 9306, 'OtherFetchStall': 6724, 'TrapStall': 0, 'IntStall': 0, 'BpStall': 8070, 'DTlbStall': 0, 'ITlbStall': 0, 'IcacheStall': 96517, 'NoStall': 720349, 'ipc': 0.752248, 'Insts': 663430, 'Cycles': 881930, 'point': '0', 'workl

## 4. 查看 Cache MPKI数据

在这里，我们采用的是一个已经运行完成的 SPEC06 Simpoint 测试结果。你可以在 `data` 目录下找到相关的文件。

In [6]:
%%bash
#!/usr/bin/env bash

pushd ../../ >/dev/null && source env.sh >/dev/null && popd >/dev/null

pushd gem5_data_proc && \
mkdir -p results && \
export PYTHONPATH=`pwd` && \
ulimit -n 65535 && \
python3 batch.py -s ../data/xs-model-l1bank -o gem5-cache-example.csv --cache&& \
python3 simpoint_cpt/compute_weighted.py \
    -r gem5-cache-example.csv \
    -j ../data/xs-model-l1bank/cluster-0-0.json \
    -o weighted.csv && \
popd


~/tutorial-2025/tutorial/p7-xs-gem5/gem5_data_proc ~/tutorial-2025/tutorial/p7-xs-gem5
workload: gcc_166, point: 23, segments: 1
[('gcc_166_23', '../data/xs-model-l1bank/gcc_166_23/m5out/stats.txt'), ('bzip2_html_10473', '../data/xs-model-l1bank/bzip2_html_10473/m5out/stats.txt'), ('gcc_expr2_6904', '../data/xs-model-l1bank/gcc_expr2_6904/m5out/stats.txt'), ('gobmk_nngs_4348', '../data/xs-model-l1bank/gobmk_nngs_4348/m5out/stats.txt'), ('gcc_200_2872', '../data/xs-model-l1bank/gcc_200_2872/m5out/stats.txt'), ('wrf_13566', '../data/xs-model-l1bank/wrf_13566/m5out/stats.txt'), ('gamess_triazolium_193880', '../data/xs-model-l1bank/gamess_triazolium_193880/m5out/stats.txt'), ('sjeng_66175', '../data/xs-model-l1bank/sjeng_66175/m5out/stats.txt'), ('h264ref_foreman.main_7969', '../data/xs-model-l1bank/h264ref_foreman.main_7969/m5out/stats.txt'), ('namd_13387', '../data/xs-model-l1bank/namd_13387/m5out/stats.txt'), ('xalancbmk_31957', '../data/xs-model-l1bank/xalancbmk_31957/m5out/stats.txt')

## 5. SPEC06 算分

在这里，我们采用的是一个已经运行完成的 SPEC06 Simpoint 测试结果。你可以在 `data` 目录下找到相关的文件。

In [7]:
%%bash

#!/usr/bin/env bash

pushd ../../ >/dev/null && source env.sh >/dev/null && popd >/dev/null

pushd gem5_data_proc && \
mkdir -p results && \
export PYTHONPATH=`pwd` && \
ulimit -n 65535 && \
python3 batch.py -s ../data/xs-model-l1bank -o gem5-score-example.csv && \
python3 simpoint_cpt/compute_weighted.py \
    -r gem5-score-example.csv \
    -j ../data/xs-model-l1bank/cluster-0-0.json \
    --score score.csv && \
popd


~/tutorial-2025/tutorial/p7-xs-gem5/gem5_data_proc ~/tutorial-2025/tutorial/p7-xs-gem5
workload: gcc_166, point: 23, segments: 1
[('gcc_166_23', '../data/xs-model-l1bank/gcc_166_23/m5out/stats.txt'), ('bzip2_html_10473', '../data/xs-model-l1bank/bzip2_html_10473/m5out/stats.txt'), ('gcc_expr2_6904', '../data/xs-model-l1bank/gcc_expr2_6904/m5out/stats.txt'), ('gobmk_nngs_4348', '../data/xs-model-l1bank/gobmk_nngs_4348/m5out/stats.txt'), ('gcc_200_2872', '../data/xs-model-l1bank/gcc_200_2872/m5out/stats.txt'), ('wrf_13566', '../data/xs-model-l1bank/wrf_13566/m5out/stats.txt'), ('gamess_triazolium_193880', '../data/xs-model-l1bank/gamess_triazolium_193880/m5out/stats.txt'), ('sjeng_66175', '../data/xs-model-l1bank/sjeng_66175/m5out/stats.txt'), ('h264ref_foreman.main_7969', '../data/xs-model-l1bank/h264ref_foreman.main_7969/m5out/stats.txt'), ('namd_13387', '../data/xs-model-l1bank/namd_13387/m5out/stats.txt'), ('xalancbmk_31957', '../data/xs-model-l1bank/xalancbmk_31957/m5out/stats.txt')

## 6. GEM5 添加 Prefetcher

底下是一个很大的脚本，我们重点关心 Shell 函数 `self_build_despacito_stream`

```bash
function self_build_despacito_stream() {
    if [ -f "$gem5_home/../tutorial/p7-xs-gem5/data/gem5.opt.despacito_stream" ]; then
        return
    fi
    pushd $gem5_home && \
    cd .. && git submodule update --init gem5 && cd gem5 && \
    (git branch -D add_a_new_prefetcher >/dev/null 2>&1 || true) && \
    (git checkout -b add_a_new_prefetcher || true) && \
    git am -3 ../tutorial/p7-xs-gem5/DespacitoStream.patch && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    cp build/RISCV/gem5.opt ../tutorial/p7-xs-gem5/data/gem5.opt.despacito_stream && \
    popd
}
```

在这里，我们创建了一个新的分支 `add_a_new_prefetcher`，并应用了一个 patch `DespacitoStream.patch`。这个 patch 为 GEM5 添加了一个新的预取器 `DespacitoStream`。在应用完 patch 后，我们编译了 GEM5，并将生成的可执行文件保存到 `data/gem5.opt.despacito_stream`。

在进行了一些其他整理操作后，我们运行了 mcf 切片，并计算得到最终的数据。

```bash
function diff_result() {
    pushd $gem5_home > /dev/null && \
    echo "===== Results =====" && \
    echo -n "Baseline                IPC: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream        IPC: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Baseline         L1D Misses: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream L1D Misses: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    popd > /dev/null
}
```

In [9]:
%%bash

#!/usr/bin/env bash

pushd ../../ >/dev/null && source env.sh >/dev/null && popd >/dev/null

function run_baseline() {
    gem5_opt_path=$1
    pushd $gem5_home && \
    export LD_LIBRARY_PATH=$gem5_home/ext/dramsim3/DRAMsim3:$LD_LIBRARY_PATH && \
    export GCBV_REF_SO=$gem5_home/riscv64-nemu-interpreter-c1469286ca32-so && \
    mkdir -p util/xs_scripts/mcf_baseline && \
    cd util/xs_scripts/mcf_baseline && \
    $gem5_opt_path $gem5_home/configs/example/xiangshan.py \
    --generic-rv-cpt=$gem5_home/../tutorial/p7-xs-gem5/data/mcf_12253_0.137576_.zstd \
    --gcpt-restorer=$gem5_home/../tutorial/p7-xs-gem5/data/normal-gcb-restorer.bin \
    -I 300000 && \
    popd
}

function run_with_despacito_stream() {
    gem5_opt_path=$1
    pushd $gem5_home && \
    export LD_LIBRARY_PATH=$gem5_home/ext/dramsim3/DRAMsim3:$LD_LIBRARY_PATH && \
    export GCBV_REF_SO=$gem5_home/riscv64-nemu-interpreter-c1469286ca32-so && \
    mkdir -p util/xs_scripts/mcf_despacito_stream && \
    cd util/xs_scripts/mcf_despacito_stream && \
    $gem5_opt_path $gem5_home/configs/example/xiangshan.py \
    --generic-rv-cpt=$gem5_home/../tutorial/p7-xs-gem5/data/mcf_12253_0.137576_.zstd \
    --gcpt-restorer=$gem5_home/../tutorial/p7-xs-gem5/data/normal-gcb-restorer.bin \
    -I 300000 && \
    popd
}

function diff_result() {
    pushd $gem5_home > /dev/null && \
    echo "===== Results =====" && \
    echo -n "Baseline                IPC: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream        IPC: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Baseline         L1D Misses: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream L1D Misses: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    popd > /dev/null
}

function self_build_baseline() {
    if [ -f "$gem5_home/../tutorial/p7-xs-gem5/data/gem5.opt" ]; then
        return
    fi
    pushd $gem5_home && \
    cd .. && git submodule update --init gem5 && cd gem5 && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    cp build/RISCV/gem5.opt ../tutorial/p7-xs-gem5/data/gem5.opt && \
    popd
}

function self_build_despacito_stream() {
    if [ -f "$gem5_home/../tutorial/p7-xs-gem5/data/gem5.opt.despacito_stream" ]; then
        return
    fi
    pushd $gem5_home && \
    cd .. && git submodule update --init gem5 && cd gem5 && \
    (git branch -D add_a_new_prefetcher >/dev/null 2>&1 || true) && \
    (git checkout -b add_a_new_prefetcher || true) && \
    git am -3 ../tutorial/p7-xs-gem5/DespacitoStream.patch && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    cp build/RISCV/gem5.opt ../tutorial/p7-xs-gem5/data/gem5.opt.despacito_stream && \
    popd
}

self_build_baseline && self_build_despacito_stream && \
run_baseline `realpath data/gem5.opt` && \
run_with_despacito_stream `realpath data/gem5.opt.despacito_stream` && \
diff_result


~/tutorial-2025/gem5 ~/tutorial-2025/tutorial/p7-xs-gem5
gem5 Simulator System.  https://www.gem5.org
gem5 is copyrighted software; use the --copyright option for details.

gem5 version [DEVELOP-FOR-22.1]
gem5 compiled Jul 12 2025 18:19:48
gem5 started Jul 13 2025 23:29:12
gem5 executing on halo, pid 1519671
command line: /home/cyy/tutorial-2025/tutorial/p7-xs-gem5/data/gem5.opt /home/cyy/tutorial-2025/gem5/configs/example/xiangshan.py --generic-rv-cpt=/home/cyy/tutorial-2025/gem5/../tutorial/p7-xs-gem5/data/mcf_12253_0.137576_.zstd --gcpt-restorer=/home/cyy/tutorial-2025/gem5/../tutorial/p7-xs-gem5/data/normal-gcb-restorer.bin -I 300000

info: Standard input is not a terminal, disabling listeners.
Obtained ref_so from GCBV_REF_SO:  /home/cyy/tutorial-2025/gem5/riscv64-nemu-interpreter-c1469286ca32-so
Obtained gcpt_restorer from args.gcpt_restorer:  /home/cyy/tutorial-2025/gem5/../tutorial/p7-xs-gem5/data/normal-gcb-restorer.bin
Simulating single core without RVV, demanding GCPT restor

warn: No dot file generated. Please install pydot to generate the dot file and pdf.


build/RISCV/arch/riscv/bare_metal/fs_workload.cc:60: info: No bootload provided, because using XS GCPT, reset to 0x80000000
build/RISCV/cpu/o3/issue_queue.cc:166: warn: load0: Use one selector by multiple identical fus
build/RISCV/cpu/o3/issue_queue.cc:166: warn: load1: Use one selector by multiple identical fus
build/RISCV/cpu/o3/issue_queue.cc:166: warn: load2: Use one selector by multiple identical fus
build/RISCV/cpu/o3/issue_queue.cc:166: warn: store0: Use one selector by multiple identical fus
build/RISCV/cpu/o3/issue_queue.cc:166: warn: store1: Use one selector by multiple identical fus
build/RISCV/cpu/o3/issue_queue.cc:166: warn: std0: Use one selector by multiple identical fus
build/RISCV/cpu/o3/issue_queue.cc:166: warn: std1: Use one selector by multiple identical fus
build/RISCV/cpu/o3/issue_queue.cc:166: warn: fpIQ2: Use one selector by multiple identical fus
build/RISCV/cpu/o3/issue_queue.cc:166: warn: vecIQ0: Use one selector by multiple identical fus
build/RISCV/cpu/o3/i

Using /home/cyy/tutorial-2025/gem5/riscv64-nemu-interpreter-c1469286ca32-so for difftest


build/RISCV/cpu/base.cc:228: warn: Difftest is enabled with ref so: /home/cyy/tutorial-2025/gem5/riscv64-nemu-interpreter-c1469286ca32-so.
build/RISCV/cpu/o3/cpu.cc:236: warn: Setting isa ptr of cpu to 0x55d5e6530a20
build/RISCV/base/statistics.hh:281: warn: One of the stats is a legacy stat. Legacy stat is a stat that does not belong to any statistics::Group. Legacy stat is deprecated.
build/RISCV/base/remote_gdb.cc:381: warn: Sockets disabled, not accepting gdb connections


...

Registering probe listeners for BaseO3CPU system.cpu
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher
system.cpu.dcache.prefetcher addTLB system.cpu.mmu.dtb
system.cpu.dcache.prefetcher addHintDownStream system.l2_caches.prefetcher
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher.berti
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher.bop_large
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher.bop_learned
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher.bop_small
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher.cmc
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher.ipcp
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher.opt
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher.spp
Registering probe listeners for Prefetcher system.cpu.dcache.prefetcher.sstride
Registering probe listeners for Pr

build/RISCV/mem/physical.cc:509: warn: Unserializing physical memory from file /home/cyy/tutorial-2025/gem5/../tutorial/p7-xs-gem5/data/mcf_12253_0.137576_.zstd
build/RISCV/mem/physical.cc:766: warn: Read zstd file size 278868649
build/RISCV/mem/physical.cc:830: warn: Total write non-zero bytes: 1595446568
build/RISCV/mem/physical.cc:711: warn: Overriding Gcpt restorer
build/RISCV/mem/physical.cc:712: warn: gCptRestorerPath: /home/cyy/tutorial-2025/gem5/../tutorial/p7-xs-gem5/data/normal-gcb-restorer.bin
build/RISCV/mem/physical.cc:727: warn: Gcpt restorer file size 4352 is larger than limit 1792, is partially loaded
build/RISCV/mem/physical.cc:735: warn: gcpt restore size: 1792
build/RISCV/sim/system.cc:561: info: Restored from Xiangshan RISC-V Checkpoint


**** REAL SIMULATION ****


build/RISCV/sim/simulate.cc:194: info: Entering event queue @ 0.  Starting simulation...
build/RISCV/cpu/base.cc:1464: warn: Start memcpy to NEMU from 0x7fb4ef400000, size=8589934592
build/RISCV/cpu/base.cc:1467: warn: Start regcpy to NEMU


Exiting @ tick 274735656 because a thread reached the max instruction count
~/tutorial-2025/tutorial/p7-xs-gem5
===== Results =====
Baseline                IPC: 0.308042
Despactio Stream        IPC: 0.363624
Baseline         L1D Misses: 203950
Despactio Stream L1D Misses: 137700
